In [2]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset("EricWiener/llm4html-descgen")

# 查看数据集的字段
print(dataset["train"].column_names)

D:\conda_envs\finalproject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['snippet', 'description']


In [4]:
from collections import Counter
import random
from sklearn.metrics import accuracy_score
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# OpenAI API Key (use your own key here)
open_ai_key = "sk-rPv5SuIO8ZdTpidSE861A965827840D587Ba6eE66556Bb1c"

# Initialize OpenAI LLM via LangChain
llm = OpenAI(
    openai_api_key=open_ai_key,
    model="gpt-4-turbo",
    base_url="https://api.aigc369.com/v1"
)

# Count the frequency of each category in the training set
category_counts = Counter([item["description"] for item in dataset["train"]])

# Select the top 10 most frequent categories from the training set
train_top_categories = [category for category, _ in category_counts.most_common(10)]

# Randomly select one example from each of the top categories for the few-shot prompt
few_shot_examples = []
for item in dataset["train"]:
    if item["description"] in train_top_categories:
        few_shot_examples.append(f'Snippet: "{item["snippet"]}"\nDescription: "{item["description"]}"')
        # Only take one example per category
        train_top_categories.remove(item["description"])
    if not train_top_categories:
        break

# Ensure few-shot examples are non-empty
if not few_shot_examples:
    raise ValueError("No valid few-shot examples were generated. Check the dataset and filtering conditions.")

# Create the few-shot prompt template
few_shot_prompt = """
Here are some example descriptions for HTML snippets:

{examples}

Now, classify this snippet:
Snippet: "{snippet}"
Description:
"""

# Create the prompt template and chain
prompt_template = PromptTemplate(input_variables=["examples", "snippet"], template=few_shot_prompt)
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Define a classification function using LangChain
def classify_with_langchain(snippet):
    """
    Classifies a given snippet using few-shot examples and the GPT-4 LLM.
    """
    # Format the few-shot examples and generate a classification
    examples = "\n".join(few_shot_examples)
    result = llm_chain.run({"examples": examples, "snippet": snippet})
    return result.strip()

# Filter test samples to match the top 10 categories from the training set
filtered_test_samples = [
    {"snippet": snippet, "description": description}
    for snippet, description in zip(dataset["test"]["snippet"], dataset["test"]["description"])
    if description in [category for category, _ in category_counts.most_common(10)]
]

# Extract filtered snippets and true labels
sampled_snippets = [sample["snippet"] for sample in filtered_test_samples]
true_labels = [sample["description"] for sample in filtered_test_samples]

# Classify each filtered test sample
predicted_labels = [classify_with_langchain(snippet) for snippet in sampled_snippets]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Few-shot model accuracy (filtered test samples matching train's top 10 categories): {accuracy * 100:.2f}%")

C:\Users\Xuewen Sun\AppData\Local\Temp\ipykernel_9708\71451280.py:13: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(
C:\Users\Xuewen Sun\AppData\Local\Temp\ipykernel_9708\71451280.py:52: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)
C:\Users\Xuewen Sun\AppData\Local\Temp\ipykernel_9708\71451280.py:61: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"examples": examples, "snippet": sni

AuthenticationError: Error code: 401 - {'error': {'message': '[sk-rPv***b1c] 该令牌额度已用尽 !token.UnlimitedQuota && token.RemainQuota = -50991 (request id: 2024120116140721562834102EUqFYV)', 'type': 'voapi_error'}}

In [8]:
from collections import Counter
import random
from sklearn.metrics import accuracy_score
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# OpenAI API Key (use your own key here)
open_ai_key = "sk-rPv5SuIO8ZdTpidSE861A965827840D587Ba6eE66556Bb1c"

# Initialize OpenAI LLM via LangChain
llm = OpenAI(
    openai_api_key=open_ai_key,
    model="gpt-4-turbo",
    base_url="https://api.aigc369.com/v1"
)

# Count the frequency of each category in the training set
category_counts = Counter([item["description"] for item in dataset["train"]])

# Select the top 10 most frequent categories from the training set
train_top_categories = [category for category, _ in category_counts.most_common(10)]

# Randomly select one example from each of the top categories for the few-shot prompt
few_shot_examples = []
for item in dataset["train"]:
    if item["description"] in train_top_categories:
        few_shot_examples.append(f'Snippet: "{item["snippet"]}"\nDescription: "{item["description"]}"')
        # Only take one example per category
        train_top_categories.remove(item["description"])
    if not train_top_categories:
        break

# Ensure few-shot examples are non-empty
if not few_shot_examples:
    raise ValueError("No valid few-shot examples were generated. Check the dataset and filtering conditions.")

# Create the few-shot prompt template
few_shot_prompt = """
Here are some example descriptions for HTML snippets:

{examples}

Now, classify this snippet:
Snippet: "{snippet}"
Description:
"""

# Create the prompt template and chain
prompt_template = PromptTemplate(input_variables=["examples", "snippet"], template=few_shot_prompt)
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Define a classification function using LangChain
def classify_with_langchain(snippet):
    """
    Classifies a given snippet using few-shot examples and the GPT-4 LLM.
    """
    # Format the few-shot examples and generate a classification
    examples = "\n".join(few_shot_examples)
    result = llm_chain.run({"examples": examples, "snippet": snippet})
    return result.strip()

# Filter test samples to match the top 10 categories from the training set
filtered_test_samples = [
    {"snippet": snippet, "description": description}
    for snippet, description in zip(dataset["test"]["snippet"], dataset["test"]["description"])
    if description in [category for category, _ in category_counts.most_common(10)]
]

# Extract filtered snippets and true labels
sampled_snippets = [sample["snippet"] for sample in filtered_test_samples]
true_labels = [sample["description"] for sample in filtered_test_samples]

# Classify each filtered test sample
predicted_labels = [classify_with_langchain(snippet) for snippet in sampled_snippets]

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Few-shot model accuracy (filtered test samples matching train's top 10 categories): {accuracy * 100:.2f}%")

Few-shot model accuracy (filtered test samples matching train's top 10 categories): 73.91%
